In [1]:
import requests
import os
from dotenv import load_dotenv
def get_access_token(url, client_id, client_secret):
    url = url
    data = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    response = requests.post(url, data=data)
    response_data = response.json()
    if response.status_code != 200:
        print('Error:', response_data['error'])
        return None
    else:
        print('Access Token:', response_data['access_token'])
        return response_data['access_token']
load_dotenv('3510.env')
client_id = os.getenv('GENIUS_CLIENT_ID')
client_secret = os.getenv('GENIUS_SECRET')
access_token = get_access_token(url='https://api.genius.com/oauth/token', client_id=client_id, client_secret=client_secret)

Access Token: ISsvY5L6Ev-qmCX5u2jgWC4BpJbn0-cDqbaEgf-SgukJWIvKwdZh4yS1g1kEff__


In [2]:
url = 'https://api.genius.com/search'
headers = {
    'Authorization': f'Bearer {access_token}'
}
params = {
    'q': 'Heaven Knows I’m Miserable Now',
}
response = requests.get(url, headers=headers, params=params).json()

In [3]:
id = response['response']['hits'][0]['result']['id']
url = f'https://api.genius.com/songs/{id}'
songRequest = requests.get(url, headers=headers).json()
lyricsurl = songRequest['response']['song']['url']
print(lyricsurl)


https://genius.com/The-smiths-heaven-knows-im-miserable-now-lyrics


In [4]:
from bs4 import BeautifulSoup
import requests

def scrape_lyrics_from_url(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        lyrics_div = soup.find('div', class_='lyrics') or soup.find_all('div', {'data-lyrics-container': 'true'})
        
        if lyrics_div:
            lyrics = "\n".join([div.get_text() for div in lyrics_div])
            return lyrics
        else:
            return "Lyrics not found on the page."
    else:
        print(f"Error fetching the song page: {response.status_code}")
        return None

lyrics = scrape_lyrics_from_url(lyricsurl)
print(lyrics)

[Verse 1]I was happy in the haze of a drunken hourBut heaven knows I'm miserable nowI was looking for a job and then I found a jobAnd heaven knows I'm miserable now[Chorus]In my life, why do I give valuable timeTo people who don't careIf I live or die?[Verse 2]Two lovers entwined pass me byAnd heaven knows I'm miserable nowI was looking for a job and then I found a jobAnd heaven knows I'm miserable now[Chorus]In my life, oh, why do I give valuable timeTo people who don't careIf I live or I die?[Verse 3]What she asked of me at the end of the dayCaligula would have blushed"Oh, you've been in the house too long," she saidAnd I naturally fled
[Chorus]In my life, why do I smileAt people who I'd much ratherKick in the eye?[Verse 4]I was happy in the haze of a drunken hourBut heaven knows I'm miserable now"Oh, you've been in the house too long," she saidAnd I naturally fled[Chorus]In my life, oh, why do I give valuable timeTo people who don't careIf I live or I die?


In [22]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

pattern = r'\[(.*?)\]'
lyrics = re.sub(pattern, '. ', lyrics)

# Tokenize the lyrics sentence-by-sentence
sentences = sent_tokenize(lyrics)
sentence_by_sentence_tokens = [word_tokenize(sentence) for sentence in sentences]
sentence_by_sentence_pos = [pos_tag(sentence) for sentence in sentence_by_sentence_tokens]

# Tokenize all at once
all_at_once_tokens = word_tokenize(lyrics)
all_at_once_pos = pos_tag(all_at_once_tokens)

# Function to map NLTK POS tag to WordNet POS tag
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

# Lemmatize sentence-by-sentence
sentence_by_sentence_lemmatized = []
for sentence in sentence_by_sentence_pos:
    lemmatized_sentence = []
    for word, tag in sentence:
        wordnet_pos = get_wordnet_pos(tag) or wordnet.NOUN
        lemmatized_word = lemmatizer.lemmatize(word, wordnet_pos)
        lemmatized_sentence.append(lemmatized_word)
    sentence_by_sentence_lemmatized.append(lemmatized_sentence)

# Lemmatize all at once
all_at_once_lemmatized = []
for word, tag in all_at_once_pos:
    wordnet_pos = get_wordnet_pos(tag) or wordnet.NOUN
    lemmatized_word = lemmatizer.lemmatize(word, wordnet_pos)
    all_at_once_lemmatized.append(lemmatized_word)

print("Tokenization and POS tagging sentence by sentence:")
for sentence in sentence_by_sentence_pos:
    print(sentence)

print("\nTokenization and POS tagging all at once:")
print(all_at_once_pos)

print("\nLemmatized sentence by sentence:")
for sentence in sentence_by_sentence_lemmatized:
    print(sentence)

print("\nLemmatized all at once:")
print(all_at_once_lemmatized)


Tokenization and POS tagging sentence by sentence:
[('I', 'PRP'), ('was', 'VBD'), ('happy', 'JJ'), ('in', 'IN'), ('the', 'DT'), ('haze', 'NN'), ('of', 'IN'), ('a', 'DT'), ('drunken', 'JJ'), ('hourBut', 'NN'), ('heaven', 'NN'), ('knows', 'VBZ'), ('I', 'PRP'), ("'m", 'VBP'), ('miserable', 'JJ'), ('nowI', 'NN'), ('was', 'VBD'), ('looking', 'VBG'), ('for', 'IN'), ('a', 'DT'), ('job', 'NN'), ('and', 'CC'), ('then', 'RB'), ('I', 'PRP'), ('found', 'VBD'), ('a', 'DT'), ('jobAnd', 'NN'), ('heaven', 'NN'), ('knows', 'VBZ'), ('I', 'PRP'), ("'m", 'VBP'), ('miserable', 'JJ'), ('nowIn', 'NN'), ('my', 'PRP$'), ('life', 'NN'), (',', ','), ('why', 'WRB'), ('do', 'VBP'), ('I', 'PRP'), ('give', 'VB'), ('valuable', 'JJ'), ('timeTo', 'JJ'), ('people', 'NNS'), ('who', 'WP'), ('do', 'VBP'), ("n't", 'RB'), ('careIf', 'VB'), ('I', 'PRP'), ('live', 'VBP'), ('or', 'CC'), ('die', 'VB'), ('?', '.'), ('Two', 'CD'), ('lovers', 'NNS'), ('entwined', 'VBD'), ('pass', 'IN'), ('me', 'PRP'), ('byAnd', 'VB'), ('heaven', 'J

In [23]:
import spacy

# Load the English language model
nlp = spacy.load('en_core_web_sm')

# Process the text
doc = nlp(lyrics)

# Lemmatize each token in the text
for token in doc:
    print(f"{token.text} -> {token.lemma_}")


I -> I
was -> be
happy -> happy
in -> in
the -> the
haze -> haze
of -> of
a -> a
drunken -> drunken
hourBut -> hourBut
heaven -> heaven
knows -> know
I -> I
'm -> be
miserable -> miserable
nowI -> nowI
was -> be
looking -> look
for -> for
a -> a
job -> job
and -> and
then -> then
I -> I
found -> find
a -> a
jobAnd -> jobAnd
heaven -> heaven
knows -> know
I -> I
'm -> be
miserable -> miserable
nowIn -> nowin
my -> my
life -> life
, -> ,
why -> why
do -> do
I -> I
give -> give
valuable -> valuable
timeTo -> timeto
people -> people
who -> who
do -> do
n't -> not
careIf -> careif
I -> I
live -> live
or -> or
die?Two -> die?two
lovers -> lover
entwined -> entwine
pass -> pass
me -> I
byAnd -> byAnd
heaven -> heaven
knows -> know
I -> I
'm -> be
miserable -> miserable
nowI -> nowI
was -> be
looking -> look
for -> for
a -> a
job -> job
and -> and
then -> then
I -> I
found -> find
a -> a
jobAnd -> jobAnd
heaven -> heaven
knows -> know
I -> I
'm -> be
miserable -> miserable
nowIn -> nowin
my ->